In [2]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
# 克隆 Face Mask Detection 資料庫
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

# 查看資料夾結構
!ls Face-Mask-Detection
!ls Face-Mask-Detection/dataset


Cloning into 'Face-Mask-Detection'...
remote: Enumerating objects: 4590, done.
remote: Total 4590 (delta 0), reused 0 (delta 0), pack-reused 4590 (from 1)
Receiving objects: 100% (4590/4590), 186.72 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (271/271), done.
Updating files: 100% (4155/4155), done.
 app.py		      detect_mask_image.py   mask_detector.model   requirements.txt
 CITATION.cff	      detect_mask_video.py   model2onnx.py	   ResNet50_v2
 CODE_OF_CONDUCT.md   face_detector	     plot.png		   search.py
 _config.yml	      images		     __pycache__	   train_mask_detector.py
 CONTRIBUTING.md      incep_v3_mask_model    Readme_images	   Windows_guide.md
 css		      LICENSE		    'README - Korean.md'
 dataset	      Logo		     README.md
with_mask  without_mask


In [3]:
# 創建暫存目錄
!mkdir tmp

# 下載 Rock-Paper-Scissors 訓練數據集
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/rps.zip \
    -O ./tmp/rps.zip

# 解壓
!unzip ./tmp/rps.zip -d ./tmp/


--2024-12-22 08:48:53--  https://storage.googleapis.com/learning-datasets/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.207, 64.233.170.207, 142.251.175.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘./tmp/rps.zip’

./tmp/rps.zip       100%[===================>] 191.38M  17.7MB/s    in 12s     

2024-12-22 08:49:06 (15.8 MB/s) - ‘./tmp/rps.zip’ saved [200682221/200682221]

Archive:  ./tmp/rps.zip
   creating: ./tmp/rps/paper/
  inflating: ./tmp/rps/paper/paper01-000.png  
  inflating: ./tmp/rps/paper/paper01-001.png  
  inflating: ./tmp/rps/paper/paper01-002.png  
  inflating: ./tmp/rps/paper/paper01-003.png  
  inflating: ./tmp/rps/paper/paper01-004.png  
  inflating: ./tmp/rps/paper/paper01-005.png  
  inflating: ./tmp/rps/paper/paper01-006.png  
  inflating: ./tmp/rps/paper/paper01-007.png  
  

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 設定訓練和測試數據的目錄
train_dir = 'Face-Mask-Detection/dataset'
test_dir = 'Face-Mask-Detection/dataset'

# 使用 ImageDataGenerator 將數據進行增強處理
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# 載入訓練集和驗證集
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 3274 images belonging to 2 classes.
Found 818 images belonging to 2 classes.


In [6]:
# 匯入所需模組
import requests
from PIL import Image
import io
from transformers import pipeline

# 初始化 Hugging Face 的圖片分類模型
classifier = pipeline("image-classification", model="google/vit-base-patch16-224")

# 輸入圖片 URL
image_url = input("請輸入圖片 URL: ")

# 下載並處理圖片
response = requests.get(image_url)
img = Image.open(io.BytesIO(response.content))

# 使用 Hugging Face 模型進行分類
results = classifier(img)

# 輸出分類結果
for result in results:
    print(f"Label: {result['label']}, Score: {result['score']:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cuda:0


請輸入圖片 URL: https://na.cx/i/eqzQJYw.jpg
Label: ski mask, Score: 0.4208
Label: mask, Score: 0.1889
Label: brassiere, bra, bandeau, Score: 0.0910
Label: gasmask, respirator, gas helmet, Score: 0.0806
Label: neck brace, Score: 0.0345


In [8]:
# 篩選關鍵標籤
keywords = ["mask"]
filtered_results = [res for res in results if any(keyword in res['label'].lower() for keyword in keywords)]

# 輸出篩選後的結果
if filtered_results:
    best_result = max(filtered_results, key=lambda x: x['score'])
    print(f"Best Relevant Label: {best_result['label']}, Score: {best_result['score']:.4f}")
else:
    print("No relevant labels found.")


Best Relevant Label: ski mask, Score: 0.4208


In [9]:
# 顯示排序後的所有相關結果（可選）
sorted_results = sorted(filtered_results, key=lambda x: x['score'], reverse=True)
print("\nSorted Relevant Results:")
for result in sorted_results:
    print(f"Label: {result['label']}, Score: {result['score']:.4f}")


Sorted Relevant Results:
Label: ski mask, Score: 0.4208
Label: mask, Score: 0.1889
Label: gasmask, respirator, gas helmet, Score: 0.0806
